## Imports

In [47]:
# Read json file
import json
import numpy as np
import requests
import os

## Get playlist from `spotify_million_playlist_dataset`

Get only the playlist name and tracks

In [48]:
playlists = []

for i in range(0, 30000, 1000):
    with open(f'spotify_million_playlist_dataset/data/mpd.slice.{i}-{i+999}.json') as f:
        data = json.load(f)
    playlists.extend([[playlist['name'], playlist['tracks']] for playlist in data['playlists']])

len(playlists)

30000

## Get all tracks' features

Using the Spotify API

In [49]:
from dotenv import load_dotenv
load_dotenv()
token = os.getenv('SPOTIFY_TOKEN')

In [64]:
def get_audio_features(track_uris):
    return requests.get(
        "https://api.spotify.com/v1/audio-features?ids=" + \
            ",".join([uri.split(':')[-1] for uri in  track_uris[:100]]),
        headers={"Authorization": f"Bearer {token}"}
    ).json()['audio_features']

In [51]:
track_features = {}

In [52]:
for playlist_title, playlist_tracks in playlists:
    for track in playlist_tracks:
        if track['track_uri'] not in track_features:
            track_features[track['track_uri']] = False

print(len(track_features))

337684


In [53]:
track_uris = [uri for uri, value in track_features.items() if value == False]
print(len(track_uris))

337684


In [87]:
try:
    for i in range(200100, len(track_uris), 100):

        batch = track_uris[i:i+100]
        track_features.update(
            {uri: features for uri, features in zip(batch, get_audio_features(batch))})
        
        print(f"{i} / {len(track_uris)}")
        
except Exception as e:
    print(e)
    print(f"{i} failed")

200100 / 337684
200200 / 337684
200300 / 337684
200400 / 337684
200500 / 337684
200600 / 337684
200700 / 337684
200800 / 337684
200900 / 337684
201000 / 337684
201100 / 337684
201200 / 337684
201300 / 337684
201400 / 337684
201500 / 337684
201600 / 337684
201700 / 337684
201800 / 337684
201900 / 337684
202000 / 337684
202100 / 337684
202200 / 337684
202300 / 337684
202400 / 337684
202500 / 337684
202600 / 337684
202700 / 337684
202800 / 337684
202900 / 337684
203000 / 337684
203100 / 337684
203200 / 337684
203300 / 337684
203400 / 337684
203500 / 337684
203600 / 337684
203700 / 337684
203800 / 337684
203900 / 337684
204000 / 337684
204100 / 337684
204200 / 337684
204300 / 337684
204400 / 337684
204500 / 337684
204600 / 337684
204700 / 337684
204800 / 337684
204900 / 337684
205000 / 337684
205100 / 337684
205200 / 337684
205300 / 337684
205400 / 337684
205500 / 337684
205600 / 337684
205700 / 337684
205800 / 337684
205900 / 337684
206000 / 337684
206100 / 337684
206200 / 337684
206300 /

In [137]:
assert all(len(track)==18 for track in track_features.values())

In [138]:
json.dump(track_features, open('track_features.json', 'w'))

## Clean the data

### Remove unnecessary columns of track features

In [139]:
cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness',
        'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo',
        'duration_ms', 'time_signature']

In [140]:
track_features_clean = {uri: {col: track[col] for col in cols} 
                        for uri, track in track_features.items()}

In [141]:
json.dump(track_features_clean, open('track_features_clean.json', 'w'))

### Reformat playlists

## Get playlist dataframe